In [4]:
from pathlib import Path
import pandas as pd
from ai_cdss.processing import ClinicalSubscales, ProtocolToClinicalMapper, compute_ppf, merge_data, compute_protocol_similarity
import shutil
from ai_cdss.ppf import load_patient_subscales, load_protocol_attributes


In [2]:

# Default data directory
DEFAULT_DATA_DIR = Path.home() / ".ai_cdss" / "data"
DEFAULT_DATA_DIR.mkdir(parents=True, exist_ok=True)


In [54]:

import numpy as np
def feature_contributions(df_A, df_B):
    # Convert to numpy
    A = df_A.to_numpy()
    B = df_B.to_numpy()

    # Compute row-wise norms
    A_norms = np.linalg.norm(A, axis=1, keepdims=True)
    B_norms = np.linalg.norm(B, axis=1, keepdims=True)
    
    # Replace zero norms with a small value to avoid NaN (division by zero)
    A_norms[A_norms == 0] = 1e-10
    B_norms[B_norms == 0] = 1e-10

    # Normalize each row to unit vectors
    A_norm = A / A_norms
    B_norm = B / B_norms

    # Compute feature contributions
    contributions = A_norm[:, np.newaxis, :] * B_norm[np.newaxis, :, :]

    return contributions

def compute_ppf(patient_deficiency, protocol_mapped):
    """ Compute the patient-protocol feature matrix (PPF) and feature contributions.
    """
    contributions = feature_contributions(patient_deficiency, protocol_mapped)
    # ppf = np.sum(contributions, axis=2)
    # ppf = pd.DataFrame(ppf, index=patient_deficiency.index, columns=protocol_mapped.index)

    # contributions = pd.DataFrame(contributions.tolist(), index=patient_deficiency.index, columns=protocol_mapped.index)
    
    # ppf_long = ppf.stack().reset_index()
    # ppf_long.columns = ["PATIENT_ID", "PROTOCOL_ID", "PPF"]

    # contrib_long = contributions.stack().reset_index()
    # contrib_long.columns = ["PATIENT_ID", "PROTOCOL_ID", "CONTRIB"]

    # return ppf_long, contrib_long
    return protocol_mapped.columns, contributions

In [55]:
# Get platform-appropriate application data directory
output_dir = Path.home() / ".ai_cdss" / "output"
output_dir.mkdir(parents=True, exist_ok=True)

patient = load_patient_subscales()
protocol = load_protocol_attributes()

patient_deficiency = ClinicalSubscales().compute_deficit_matrix(patient)
protocol_mapped    = ProtocolToClinicalMapper().map_protocol_features(protocol)

ppf, contrib = compute_ppf(patient_deficiency, protocol_mapped)
# ppf_contrib = merge_data(ppf, contrib)
# ppf_contrib.set_index('PATIENT_ID', inplace=True)